```markdown
### Introduction to Earthaccess and TEMPO Data

In this section, we utilize the `earthaccess` module, a Python library designed to simplify access to NASA Earthdata resources. This library provides tools for authentication, data search, and downloading Earth science datasets.

We specifically focus on the **TEMPO (Tropospheric Emissions: Monitoring of Pollution)** dataset, which provides Level 3 NO2 data. This dataset is valuable for monitoring air quality and studying atmospheric pollution. Using `earthaccess`, we authenticate with NASA Earthdata, search for the TEMPO NO2 dataset within a specified temporal range, and download the data for further analysis.
```

In [4]:
# Install required packages
!pip install ipyleaflet ipywidgets --quiet

# Import libraries
from ipyleaflet import Map, WMSLayer, LayersControl
from ipywidgets import VBox, Dropdown, Label, HBox
from datetime import datetime, timedelta
import pytz

# Create the interactive map
m = Map(center=(39.8283, -98.5795), zoom=4)

# WMS service URL
wms_url = (
    "https://gis.earthdata.nasa.gov/image/services/"
    "C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN/"
    "ImageServer/WMSServer?"
)

# Generate hourly timestamps from Nov 7 to Nov 10, 2024
timestamps = []
start_time = datetime(2024, 11, 7, 0, 0, tzinfo=pytz.UTC)
end_time = datetime(2024, 11, 10, 23, 0, tzinfo=pytz.UTC)
interval = timedelta(hours=1)
current = start_time
while current <= end_time:
    timestamps.append(current.strftime("%Y-%m-%dT%H:%M:%S"))
    current += interval

# Dropdown widget
dropdown = Dropdown(
    options=timestamps,
    value=timestamps[0],
    description="Time:"
)

# Label for current time
time_label = Label(value=f"Time: {dropdown.value}")

# Create initial WMS layer and hold it in a mutable container (list of one element)
current_layer = [
    WMSLayer(
        url=wms_url,
        layers="TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN:None:NO2_Troposphere",
        format="image/png",
        transparent=True,
        version="1.3.0",
        time=dropdown.value,
        name=f"NO2 @ {dropdown.value}"
    )
]

# Add initial layer to map
m.add_layer(current_layer[0])
m.add_control(LayersControl(position='topright'))

# Dropdown change handler
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        new_time = change['new']
        
        # Remove current layer
        m.remove_layer(current_layer[0])
        
        # Create and add new WMS layer
        new_layer = WMSLayer(
            url=wms_url,
            layers="TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN:None:NO2_Troposphere",
            format="image/png",
            transparent=True,
            version="1.3.0",
            time=new_time,
            name=f"NO2 @ {new_time}"
        )
        m.add_layer(new_layer)
        current_layer[0] = new_layer
        time_label.value = f"Time: {new_time}"

# Connect dropdown to handler
dropdown.observe(on_dropdown_change)

# Show UI and map
VBox([HBox([dropdown, time_label]), m])


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
# Install required packages
!pip install rioxarray xarray leafmap ipywidgets requests matplotlib --quiet


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [8]:
# Imports
import requests
import rioxarray
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import leafmap
from ipywidgets import Dropdown, VBox, Label, HBox, Output
from datetime import datetime, timedelta
import pytz
from IPython.display import display

# WCS URL and coverage
base_url = (
    "https://gis.earthdata.nasa.gov/image/services/"
    "C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN/"
    "ImageServer/WCSServer"
)
coverage_id = "_vsis3_prd-arcgis-data_TEMPO_TEMPO_NO2_L3_V03_TEMPO_NO2_L3_V03_T_B.crf"

# NYC bounding box
bbox = [-74.3, 40.4, -73.6, 41.0]  # [minx, miny, maxx, maxy]

# Time range: Nov 9–10, 2024 (UTC)
timestamps = []
start = datetime(2024, 11, 9, 0, 0, tzinfo=pytz.UTC)
end = datetime(2024, 11, 10, 23, 0, tzinfo=pytz.UTC)
while start <= end:
    timestamps.append(start.strftime("%Y-%m-%dT%H:%M:%SZ"))
    start += timedelta(hours=1)

# UI widgets
dropdown = Dropdown(options=timestamps, value=timestamps[0], description="Time:")
status_label = Label(value="Select a time to view NO₂ in NYC")
output_file = "tempo_no2_nyc.tif"
output_map = Output()

# UI layout
ui = VBox([HBox([dropdown, status_label]), output_map])

# Fetch WCS file for selected time
def fetch_wcs_data(time_value):
    params = {
        "service": "WCS",
        "version": "2.0.1",
        "request": "GetCoverage",
        "coverageId": coverage_id,
        "format": "image/tiff",
        "subset": [
            f"Lat({bbox[1]},{bbox[3]})",
            f"Long({bbox[0]},{bbox[2]})",
            f"ansi(\"{time_value}\")"
        ]
    }
    response = requests.get(base_url, params=params, stream=True)
    if response.status_code == 200:
        with open(output_file, 'wb') as f:
            for chunk in response.iter_content(8192):
                f.write(chunk)
        return True
    return False

# Render map from file
def show_map_from_file(filename):
    ds = rioxarray.open_rasterio(filename, masked=True)
    band = ds.squeeze()
    quantiles = np.nanpercentile(band, [0, 20, 40, 60, 80, 100])
    classified = xr.apply_ufunc(np.digitize, band, quantiles)
    cmap = plt.get_cmap("YlOrRd", len(quantiles))
    m = leafmap.Map(center=(40.7, -74), zoom=9)
    m.add_raster(classified, colormap=cmap, layer_name="Classified NO₂ (NYC)")
    return m

# Dropdown change handler
def update_map(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_time = change['new']
        status_label.value = f"Fetching NO₂ for {selected_time}..."
        if fetch_wcs_data(selected_time):
            status_label.value = f"Showing NO₂ for {selected_time}"
            m = show_map_from_file(output_file)
            output_map.clear_output()
            with output_map:
                display(m)
        else:
            status_label.value = "Failed to retrieve data."

# Attach handler
dropdown.observe(update_map)

# Display interface
ui


In [10]:
try:
    import earthaccess
    print("earthaccess is already installed")
except ImportError:
    !pip install earthaccess
    import earthaccess
Auth = earthaccess.login()
tempo = earthaccess.search_data(
    short_name="TEMPO_NO2_L3",
    temporal=('2024-11-08 12:00:00', '2024-11-08 23:59:59'),
    bounding_box=(-74.2591, 40.4774, -73.7004, 40.9176)  # Bounding box for NYC
)
# Display the search results
print(f"Found {len(tempo)} results")
# Print the metadata for each result
for result in tempo:
    print(result)


earthaccess is already installed
Found 10 results
Collection: {'ShortName': 'TEMPO_NO2_L3', 'Version': 'V03'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Latitude': 57.62, 'Longitude': -110.68}, {'Latitude': 53.38, 'Longitude': -108.38}, {'Latitude': 49.14, 'Longitude': -106.62}, {'Latitude': 44.12, 'Longitude': -105.02}, {'Latitude': 39.48, 'Longitude': -103.88}, {'Latitude': 34.7, 'Longitude': -102.96}, {'Latitude': 29.14, 'Longitude': -102.14}, {'Latitude': 23.68, 'Longitude': -101.54}, {'Latitude': 17.22, 'Longitude': -101.04}, {'Latitude': 17.2, 'Longitude': -82.4}, {'Latitude': 17.46, 'Longitude': -65.14}, {'Latitude': 21.36, 'Longitude': -64.28}, {'Latitude': 25.16, 'Longitude': -63.24001}, {'Latitude': 28.92, 'Longitude': -61.98}, {'Latitude': 32.64, 'Longitude': -60.46}, {'Latitude': 35.78, 'Longitude': -58.92001}, {'Latitude': 39.06, 'Longitude': -57}, {'Latitude': 42, 'Longitude': -54.94}, {'Latitude': 44.8, 'Longitude

In [11]:
earthaccess.download(tempo, 'data')

QUEUEING TASKS | :   0%|          | 0/10 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/10 [00:00<?, ?it/s]

Error while downloading the file TEMPO_NO2_L3_V03_20241108T162215Z_S007.nc
Traceback (most recent call last):
  File "C:\Users\kyttm\AppData\Roaming\Python\Python310\site-packages\urllib3\response.py", line 754, in _error_catcher
    yield
  File "C:\Users\kyttm\AppData\Roaming\Python\Python310\site-packages\urllib3\response.py", line 900, in _raw_read
    raise IncompleteRead(self._fp_bytes_read, self.length_remaining)
urllib3.exceptions.IncompleteRead: IncompleteRead(546781754 bytes read, 312039231 more expected)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\kyttm\AppData\Roaming\Python\Python310\site-packages\requests\models.py", line 820, in generate
    yield from self.raw.stream(chunk_size, decode_content=True)
  File "C:\Users\kyttm\AppData\Roaming\Python\Python310\site-packages\urllib3\response.py", line 1066, in stream
    data = self.read(amt=amt, decode_content=decode_content)
  File "C:\Users\kyttm

COLLECTING RESULTS | :   0%|          | 0/10 [00:00<?, ?it/s]

Exception: 

In [13]:
import xarray as xr

# Load the NetCDF file
file_path = 'data/TEMPO_NO2_L3_V03_20241109T134215Z_S004.nc'
data = xr.open_dataset(file_path)

# Display the dataset structure
print(data)

# List all variables in the dataset
print("Variables in the dataset:")
print(data.variables)

# Display metadata for the dataset
print("Metadata:")
print(data.attrs)

# Plot one of the variables (e.g., NO2 concentration)
if 'NO2' in data.variables:
    data['NO2'].plot()
else:
    print("NO2 variable not found in the dataset.")

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html